In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch as T
from foundations import hparams
from threat_specification.greedy_subset import load_threat_specification
from platforms import platform as P
from platforms.local import Platform as LocalPlatform
from platforms.platform import get_platform

In [4]:
get_platform() == None  

True

In [5]:
old_platform = None
P._PLATFORM = LocalPlatform()
# P._PLATFORM = old_platform

In [6]:
get_platform().runner_root

'/cis/home/rmuthuk1/nonisotropic/runner_data'

In [7]:
cifar10_dataset_hparams = hparams.DatasetHparams(
    dataset_name="cifar10",
    batch_size=256,
    num_labels=10,
    num_channels=3,
    num_spatial_dims=32,
    num_train=50000,
    num_test=10000,
)

cifar10_ts = load_threat_specification(cifar10_dataset_hparams)

cifar10_ts.shape

torch.Size([10, 50, 3, 32, 32])

In [8]:
ref_inputs = T.randn((256, 3, 32, 32), requires_grad=True).to("cuda")
ref_labels = T.randint(0, 10, (256,)).to("cuda")
perturbed_inputs = ref_inputs + 0.05*T.randn((256, 3, 32, 32), requires_grad=True).to("cuda")
N_threshold = 0.3

In [9]:
from threat_specification.projected_displacement import non_isotropic_projection as NIP

projected_inputs = NIP(
    reference_inputs=ref_inputs,
    reference_labels=ref_labels,
    perturbed_inputs=perturbed_inputs,
    greedy_subsets=cifar10_ts,
    threshold=N_threshold,
)

projected_inputs.requires_grad

Inside non-isotropic projection 1
requires_grad check
reference_inputs : True
reference_labels : False
perturbed_inputs : True
greedy_subsets : False


False

In [11]:
cifar10_ts.requires_grad

False

In [14]:
cifar10_ts.shape

torch.Size([10, 50, 3, 32, 32])

In [15]:
anchor_label = 0

In [26]:
anchor_points = cifar10_ts[0].to("cuda")
anchor_points = T.flatten(anchor_points, start_dim=1)
print(anchor_points.shape)

torch.Size([50, 3072])


In [17]:
ref_inputs_f = T.flatten(ref_inputs, start_dim=1)
p_inputs_f = T.flatten(perturbed_inputs, start_dim=1)

In [21]:
print(ref_inputs_f.shape)
print(p_inputs_f.shape)

torch.Size([256, 3072])
torch.Size([256, 3072])


In [18]:
ref_inputs_f.shape[1] == anchor_points.shape[1]

True

In [23]:
perturbations = -(ref_inputs_f - p_inputs_f).unsqueeze(1)
perturbations.shape

torch.Size([256, 1, 3072])

In [24]:
ref_inputs_f.unsqueeze(1).shape

torch.Size([256, 1, 3072])

In [28]:
unsafe_directions = -(ref_inputs_f.unsqueeze(1) - anchor_points)
unsafe_directions.shape

torch.Size([256, 50, 3072])

In [29]:
beta = 0.5

In [30]:
unsafe_norms = beta * (T.linalg.norm(unsafe_directions, dim=2, ord=2) ** 2) + 1e-2
unsafe_norms.shape

torch.Size([256, 50])

In [31]:
unsafe_directions = unsafe_directions / unsafe_norms.unsqueeze(-1)
unsafe_directions.shape

torch.Size([256, 50, 3072])

In [32]:
scaled_projections = T.bmm(perturbations, unsafe_directions.permute(0, 2, 1))
scaled_projections.shape

torch.Size([256, 1, 50])

In [35]:
scaled_projections[ref_labels == 0] = 0.0

In [36]:
cifar10_ts.dtype

torch.float32

In [37]:
T.backends.cudnn.version()

8700

In [38]:
T.backends.cudnn.enabled

True

In [40]:
T.backends.cudnn.allow_tf32

True

In [42]:
T.randperm(10)

tensor([8, 9, 5, 2, 1, 4, 0, 6, 7, 3])

In [44]:
a = T.randn(10)
print(a)
print(a[T.randperm(10)])

tensor([-0.2050, -0.0953,  0.5121, -1.7345, -0.0901, -0.6095,  0.7175,  2.4800,
         0.3540, -0.1987])
tensor([-0.0901, -0.6095, -1.7345, -0.1987,  0.5121,  0.3540, -0.0953, -0.2050,
         2.4800,  0.7175])
